In [1]:
# Required python package imports
import sqlalchemy
import sys
from sqlalchemy import create_engine, MetaData, Table, select
import pyodbc
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
import numpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time 
import multiprocessing 
%matplotlib inline
import matplotlib.image as image
import datetime
from yahoo_finance import Share

In [2]:
#tickers_ls = ['YHOO','IBM','MSFT']
tickers_tb = pd.read_csv('C:/Users/ywu/Google Drive/Future/Sample/Efficient Frontier/tickers_ef.csv')
tickers_tb = tickers_tb.set_index('Ticker')
tickers_ls=pd.Series.tolist(tickers_tb.index)

In [3]:
#Create table for price data
px_data=Share('IBM')
px_data=pd.DataFrame(Share('IBM').get_historical('2016-09-21', '2016-09-21'))
px_data=px_data[px_data.Symbol=='xxxx']

In [20]:
for ticker in tickers_ls:
    print (ticker, end="", flush=True),
    px_data_raw=pd.DataFrame(Share(ticker).get_historical('2001-09-21', '2016-09-21'))
    if len(px_data_raw)!=0:
        px_data=px_data.append(px_data_raw)
    #px_data=px_data.reindex()

JBCSPYIWMTLTIYREEMXLUGLDIBBSHY

In [22]:
px_data_bak=px_data #save a backup
#px_data=px_data_bak #reload backup
#px_data.drop(['ret_1yr','std_1yr'], axis=1, inplace=True)
#px_data.head(3)

In [23]:
px_data['Adj_Close']=px_data.Adj_Close.astype(float)
px_data=px_data.sort_values(by=['Symbol','Date'])
px_data['ret']=px_data.groupby('Symbol')['Adj_Close'].pct_change().fillna(0)
px_data['ret_tr']=px_data.ret+1
#x_data=px_data[px_data['ret'].notnull()]
px_data=px_data.join(tickers_tb, on='Symbol')
px_data.head(3)

,Adj_Close,Close,Date,High,Low,Open,Symbol,Volume,ret,ret_tr
3384,8.963368,100.999984,2003-04-14,100.999984,100.299957,100.299957,EEM,93600,0.000000,1.000000
3383,9.069868,102.200043,2003-04-15,102.500012,101.61999,101.61999,EEM,421200,0.011882,1.011882
3382,9.174588,103.380026,2003-04-16,103.79997,103.300023,103.300023,EEM,9000,0.011546,1.011546


In [28]:

px_data.to_csv('C:/Users/ywu/Google Drive/Future/Sample/Efficient Frontier/data.csv', index=False)

In [368]:
#summary tables
ret_1yr=pd.DataFrame(px_data.groupby('Symbol')['ret_tr'].prod()-1)
std_1yr=pd.DataFrame((px_data.groupby('Symbol')['ret'].std())*(255**0.5))
px_data_summary=pd.concat([ret_1yr,std_1yr], axis=1, join='inner')
px_data_summary.columns=['ret_1yr','std_1yr']
#join to the px table
px_data=px_data.join(px_data_summary, on='Symbol')
px_data.head(3)

,Adj_Close,Close,Date,High,Low,Open,Symbol,Volume,ret,ret_tr,ret_1yr,std_1yr
252,34.661335,35.049999,2015-09-22,35.330002,34.939999,35.189999,A,2283300,0.000000,1.000000,0.328281,0.237967
251,34.562446,34.950001,2015-09-23,35.240002,34.810001,35.099998,A,1962600,-0.002853,0.997147,0.328281,0.237967
250,34.166880,34.549999,2015-09-24,34.639999,34.150002,34.560001,A,2028900,-0.011445,0.988555,0.328281,0.237967


In [379]:
#px_data=px_data.join(tickers_tb, on='Symbol')
px_data

,Adj_Close,Close,Date,High,Low,Open,Symbol,Volume,ret,ret_tr,ret_1yr,std_1yr,Name,Sector
252,34.661335,35.049999,2015-09-22,35.330002,34.939999,35.189999,A,2283300,0.000000e+00,1.000000,0.328281,0.237967,Agilent Technologies Inc,Health Care
251,34.562446,34.950001,2015-09-23,35.240002,34.810001,35.099998,A,1962600,-2.853006e-03,0.997147,0.328281,0.237967,Agilent Technologies Inc,Health Care
250,34.166880,34.549999,2015-09-24,34.639999,34.150002,34.560001,A,2028900,-1.144497e-02,0.988555,0.328281,0.237967,Agilent Technologies Inc,Health Care
249,34.166883,34.450001,2015-09-25,35.02,34.099998,34.700001,A,2730700,8.780433e-08,1.000000,0.328281,0.237967,Agilent Technologies Inc,Health Care
248,33.095757,33.369999,2015-09-28,34.27,33.32,34.130001,A,2615400,-3.134983e-02,0.968650,0.328281,0.237967,Agilent Technologies Inc,Health Care
247,33.462719,33.740002,2015-09-29,34.060001,33.240002,33.360001,A,2252400,1.108789e-02,1.011088,0.328281,0.237967,Agilent Technologies Inc,Health Care
246,34.047870,34.330002,2015-09-30,34.43,33.669998,34.080002,A,2545700,1.748665e-02,1.017487,0.328281,0.237967,Agilent Technologies Inc,Health Care
245,33.462719,33.740002,2015-10-01,34.48,33.48,34.32,A,3011400,-1.718613e-02,0.982814,0.328281,0.237967,Agilent Technologies Inc,Health Care
244,34.385072,34.669998,2015-10-02,34.68,33.119999,33.18,A,3519400,2.756360e-02,1.027564,0.328281,0.237967,Agilent Technologies Inc,Health Care
243,35.049568,35.34,2015-10-05,35.389999,34.84,35.040001,A,1675400,1.932513e-02,1.019325,0.328281,0.237967,Agilent Technologies Inc,Health Care
